In [ ]:
!pip install gradio --quiet
!pip install timm --quiet

     |████████████████████████████████| 549 kB 5.1 MB/s 
     |████████████████████████████████| 182 kB 65.2 MB/s 


In [ ]:
import gradio as gr
import torch
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from PIL import Image

model = torch.load('model.pt',map_location ='cpu')
model.eval()

labels = ['non-tumor','tumor']
transform = create_transform(**resolve_data_config({},model = model))

def predict_fn(img):
    img = img.convert('RGB')
    img = transform(img).unsqueeze(0)

    with torch.no_grad():
        out = model(img)

    probabilites = torch.nn.functional.softmax(out[0], dim=0)

    values, indices = torch.topk(probabilites, k=int(1))

    return {labels[i]: v.item() for i, v in zip(indices, values)}

gr.Interface(fn=predict_fn, inputs=gr.inputs.Image(type='pil'), outputs='label',interpretation='default',
description = "Upload an image of an MRI and the model would predict if it is having a tumor or not").launch(debug='True')

/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:256: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
